<a href="https://colab.research.google.com/github/JohnnyCoRuyzo/Control_Inteligente/blob/New_Projects/Playing_with_a_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import keras

print("")

# CLASE DE LA CAPA DE LA RED

In [ ]:
class neural_layer():
  
  def __init__(self, n_conn, n_neur, act_f):
    self.act_f = act_f
    self.b = np.random.rand(1, n_neur) * 2 - 1
    self.W = np.random.rand(n_conn, n_neur) * 2 - 1
    
print("")

# FUNCIONES DE ACTIVACIÓN

In [ ]:
sigm = (lambda x: 1 / (1 + np.e ** (-x)),
         lambda x: x * (1 - x))
 
relu = lambda x: np.maximum(0,x)

print("")

# CREACION DE CAPAS DE LA RED

In [ ]:

def create_nn(topology, act_f):
  
  nn = []
   
  for l, layer in enumerate(topology[:-1]):

    nn.append(neural_layer(layer,topology[l+1],act_f))

  return nn

print("")

# ALGORITMO DE ENTRENAMIENTO O PREDICCION

In [ ]:
def train(neural_net, X, Y, l2_cost, lr= 0.5, train = True): 
  
  out = [(None,X)]

  # Forward pass

  for l, layer in enumerate(neural_net):

    z = out[l][1] @ neural_net[l].W + neural_net[l].b
    a = neural_net[l].act_f[0](z)

    out.append((z,a))

  if train:

    #Backward pass 
    deltas = []

    for l in reversed(range(0, len(neural_net))):

      z = out[l+1][0]
      a = out[l+1][1]

      if (l == len(neural_net) - 1):
        deltas.insert(0, l2_cost[1](a,Y) * neural_net[l].act_f[1](a))
      else:
        deltas.insert(0, deltas[0] @ _W.T * neural_net[l].act_f[1](a))
        
      _W = neural_net[l].W

      #Gradient descent
      neural_net[l].b = neural_net[l].b - np.mean(deltas[0], axis=0, keepdims=True) * lr
      neural_net[l].W = neural_net[l].W - out[l][1].T @ deltas[0] * lr

  return out[-1][1]

print("")

# CREAR EL DATASET 

In [ ]:
from sklearn.datasets import make_moons

# Numero de muestras de los datos.
nS = 500

def twospirals(n_points, noise=.5):
    """
     Returns the two spirals dataset.
    """
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise 
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise /12
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), 
            np.hstack((np.zeros(n_points),np.ones(n_points))))

X,Y = twospirals (n_points=nS, noise= 0.05)

Xt = np.zeros((2*nS,4))

Xt[:,0] = X[:,0]
Xt[:,1] = X[:,1]
Xt[:,2] = np.sin(X[:,0])
Xt[:,3] = np.sin(X[:,1])

Y = Y[:, np.newaxis]

plt.scatter(X[Y[:,0] == 0,0],X[Y[:,0] == 0,1], c="skyblue")
plt.scatter(X[Y[:,0] == 1,0],X[Y[:,0] == 1,1], c="salmon")

# ENTRENAMIENTO

In [ ]:
import time
from IPython.display import clear_output

# Numero de registros de los datos.
n = 5000

# Cantidad de caracteristicas de los datos
p = 4

# Learning Rate
lr = 0.001

topology = [p, 30, 10, 5]

neural_n = create_nn(topology,sigm)

loss = []

l2_cost = (lambda Yp, Yr: np.mean((Yp - Yr) ** 2),
           lambda Yp, Yr: (Yp - Yr))

for i in range(n):

  pY = train(neural_n, Xt, Y, l2_cost, lr)

  if i%10 == 0:

    loss.append(l2_cost[0](pY,Y))

    res = 100
    
    _x0 = np.linspace(-15, 15, res)
    _x1 = np.linspace(-15, 15, res)

    _Y = np.zeros((res,res))

    for i0, x0 in enumerate(_x0):
      for i1, x1 in enumerate(_x1):
        _Y[i0, i1] = train(neural_n, np.array([[x0, x1, np.sin(x0), np.sin(x1)]]), Y, l2_cost, train = False)[0][0]
        if (_Y[i0,i1] < 0.5):
          _Y[i0,i1] = 0
        else: 
          _Y[i0,i1] = 1

    plt.pcolormesh(_x0,_x1,_Y,cmap="coolwarm")
    plt.colorbar()
    plt.axis("equal")

    plt.scatter(X[Y[:,0] == 0, 0],X[Y[:,0] == 0, 1], c="skyblue")
    plt.scatter(X[Y[:,0] == 1, 0],X[Y[:,0] == 1, 1], c="salmon")

    clear_output(wait=True)
    plt.show()
    plt.plot(range(len(loss)), loss)
    plt.show()
    time.sleep(0.5)
     